# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch==1.4.0
import torch
print(torch.__version__)

     |████████████████████████████████| 753.4 MB 6.7 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
1.4.0


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning) 

In [ ]:
!pip install stanfordnlp
import stanfordnlp
stanfordnlp.download('en')

     |████████████████████████████████| 158 kB 16.8 MB/s 
Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)
Y

Default download directory: /root/stanfordnlp_resources
Hit enter to continue or type an alternate directory.


Download location: /root/stanfordnlp_resources/en_ewt_models.zip


100%|██████████| 235M/235M [00:41<00:00, 5.64MB/s]



Download complete.  Models saved to: /root/stanfordnlp_resources/en_ewt_models.zip
Extracting models file for: en_ewt
Cleaning up...Done.


In [ ]:
from xml.etree import cElementTree as ET
import pandas as pd
import string
import spacy
import re
import random
import sklearn
from sklearn.model_selection import train_test_split
import csv
import numpy as np

In [ ]:
root_dir = "/content/drive/MyDrive/Acads/4-1/NLP/NLP Project/Topic 5/"
train_folder = "Train/SemEval'14-ABSA-TrainData_v2 & AnnotationGuidelines/"
laptop_train_file = "Laptop_Train_v2.xml"
restaurants_train_file = "Restaurants_Train_v2.xml"
test_1_folder = "Test1/ABSA_TestData_PhaseA/ABSA_TestData_PhaseA/"
laptop_test_1_file = "Laptops_Test_Data_PhaseA.xml"
restaurants_test_1_file = "Restaurants_Test_Data_PhaseA.xml"
test_2_folder = "Test2/ABSA_TestData_PhaseB/"
laptop_test_2_file = "Laptops_Test_Data_phaseB.xml"
restaurants_test_2_file = "Restaurants_Test_Data_phaseB.xml"

# Data Extraction & Cleaning

In [ ]:
class AspectWord():
    def __init__(self, aspect_term, sentiment_terms, polarity = None):
        # Words
        self.aspect_term = aspect_term
        # List of Words
        self.sentiment_terms = sentiment_terms
        # Polarity: +,-,neutral
        self.polarity = polarity

    def __str__(self):
        return f"Aspect Term: {self.aspect_term} Sentiment Terms: {self.sentiment_terms} Polarity: {self.polarity}"
    
    def __repr__(self):
        return self.__str__()

In [ ]:
class Sentence():
    def __init__(self, sentence_id, sentence, data_type = "Train",
                 actual_sentence_id = None, actual_aspect_words = []): 
        # Int
        self.sentence_id = sentence_id
        # String
        self.sentence = sentence
        # String
        self.data_type = data_type
        # String
        self.actual_sentence_id = actual_sentence_id
        # List of AspectWord
        self.actual_aspect_words = actual_aspect_words
        # List of AspectWord
        self.generated_aspect_words = []

    def __str__(self):
        if self.data_type == "Train":
            return f"ID: {self.sentence_id} Sentence: {self.sentence} {self.actual_aspect_words}"
        else:
            return f"ID: {self.sentence_id} Sentence: {self.sentence} {self.generated_aspect_words}"
    
    def __repr__(self):
        return self.__str__()

In [ ]:
def xml_to_sentences(path,data_type = "Train"):
    data = []
    tree = ET.parse(path)
    root = tree.getroot()
    id = 1
    for page in root.findall('sentence'):
        sentence_id = id
        sentence = page[0].text
        actual_sentence_id = page.attrib["id"]
        temp = []
        for i in range(1,len(page)):
            temp.append([x.attrib for x in page[i]])
        actual_aspect_terms = []
        actual_polarity = []
        if (data_type == "Train"):
            if len(temp) > 0:
            # Ignore Sentences without aspect terms if training data
                aspect_words = []
                for x in temp[0]:
                    aspect_words.append(AspectWord(x['term'],[],x['polarity']))
                id += 1
                data.append(Sentence(sentence_id, sentence, data_type, actual_sentence_id, aspect_words))
        elif (data_type == "Test"):
            data.append(Sentence(sentence_id, sentence, data_type, actual_sentence_id))
            id += 1
        else:
            print("Incorrect Data Type")
            return None
    return data

In [ ]:
def data_list_to_df(data_list, aspect_terms = "actual", aspect_polarity = True):
    
    data = pd.DataFrame(columns = ["ID","Sentence","Actual Aspect Terms","Actual Sentiment Terms","Actual Polarities","Generated Aspect Terms","Generated Sentiment Terms","Generated Polarities"])
    
    for i in range(len(data_list)):
        data.loc[len(data.index)] = [data_list[i].sentence_id, data_list[i].sentence, [x.aspect_term for x in data_list[i].actual_aspect_words], [x.sentiment_terms for x in data_list[i].actual_aspect_words], [x.polarity for x in data_list[i].actual_aspect_words], [x.aspect_term for x in data_list[i].generated_aspect_words], [x.sentiment_terms for x in data_list[i].generated_aspect_words], [x.polarity for x in data_list[i].generated_aspect_words]] 
    
    if aspect_terms == "actual":
        data.drop(["Generated Aspect Terms","Generated Sentiment Terms","Generated Polarities","Actual Sentiment Terms"], axis = 1, inplace = True)
    elif aspect_terms == "generated":
        data.drop(["Actual Aspect Terms","Actual Sentiment Terms","Actual Polarities"], axis = 1, inplace = True)
    elif aspect_terms == "none":
        data.drop(["Generated Aspect Terms","Generated Sentiment Terms","Generated Polarities","Actual Aspect Terms","Actual Sentiment Terms","Actual Polarities"], axis = 1, inplace = True)

    if not aspect_polarity:
        try:
            data.drop(["Generated Polarities"],axis=1, inplace = True)
        except:
            pass
        try:
            data.drop(["Actual Polarities"],axis=1, inplace = True)
        except:
            pass

    return data

In [ ]:
laptop_train_data = xml_to_sentences(root_dir + train_folder + laptop_train_file)
laptop_train_data_size = len(laptop_train_data)
print(f"Total sentences: {laptop_train_data_size}")
laptop_train_data_df = data_list_to_df(laptop_train_data, aspect_terms = "actual", aspect_polarity = True)
laptop_train_data_df.head()

Total sentences: 1488


,ID,Sentence,Actual Aspect Terms,Actual Polarities
0,1,I charge it at night and skip taking the cord ...,"[cord, battery life]","[neutral, positive]"
1,2,The tech guy then said the service center does...,"[service center, ""sales"" team, tech guy]","[negative, negative, neutral]"
2,3,"it is of high quality, has a killer GUI, is ex...","[quality, GUI, applications, use]","[positive, positive, positive, positive]"
3,4,Easy to start up and does not overheat as much...,[start up],[positive]
4,5,"I even got my teenage son one, because of the ...","[features, iChat, Photobooth, garage band]","[positive, positive, positive, positive]"


In [ ]:
# Split into train and validation
np.random.seed(7)
random.seed(9)
laptop_train_data, laptop_valid_data = sklearn.model_selection.train_test_split(laptop_train_data, test_size=0.1)
print(f"Size of Training Data:{len(laptop_valid_data)}, Size of Validation Data: {len(laptop_train_data)}")

Size of Training Data:149, Size of Validation Data: 1339


# Preprocessing

In [ ]:
# Remove Punctuations - {, . "" ' ;}
# Check stopword Removal

# Find Aspect Terms

In [ ]:
nlp = stanfordnlp.Pipeline()

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand

In [ ]:
def extract_attributes(text):
    doc = nlp(text)
    parsed_text = {'word':[], 'lemma':[], 'pos':[], 'dep':[], 'gov':[], 'gov_pos':[]}
    for sent in doc.sentences:
        for wrd in sent.words:
            parsed_text['word'].append(wrd.text)
            parsed_text['lemma'].append(wrd.lemma)
            parsed_text['pos'].append(wrd.pos)
            parsed_text['dep'].append(wrd.dependency_relation)
    for sent in doc.sentences:
        for wrd in sent.words:        
            if wrd.governor == 0:
                parsed_text['gov'].append("ROOT")
                parsed_text['gov_pos'].append("-")
            else:   
                parsed_text['gov'].append(parsed_text['word'][wrd.governor-1])
                parsed_text['gov_pos'].append(parsed_text['pos'][wrd.governor-1])
    
    return pd.DataFrame(parsed_text)

In [ ]:
text = "The display on this computer is the best I've seen in a very long time, the battery life is very long and very convenient."
extract_attributes(text)

,word,lemma,pos,dep,gov,gov_pos
0,The,the,DT,det,display,NN
1,display,display,NN,nsubj,best,JJS
2,on,on,IN,case,computer,NN
3,this,this,DT,det,computer,NN
4,computer,computer,NN,nmod,display,NN
5,is,be,VBZ,cop,best,JJS
6,the,the,DT,det,best,JJS
7,best,best,JJS,root,ROOT,-
8,I,I,PRP,nsubj,seen,VBN
9,'ve,have,VBP,aux,seen,VBN


In [ ]:
text = "Bell, based in Los Angeles, makes and distributes electronic, computer and building products."
extract_attributes(text)

,word,lemma,pos,dep,gov,gov_pos
0,Bell,Bell,NNP,nsubj,makes,VBZ
1,",",",",",",punct,Bell,NNP
2,based,base,VBN,case,Angeles,NNP
3,in,in,IN,case,Angeles,NNP
4,Los,Los,NNP,compound,Angeles,NNP
5,Angeles,Angeles,NNP,nmod,Bell,NNP
6,",",",",",",punct,makes,VBZ
7,makes,make,VBZ,root,ROOT,-
8,and,and,CC,cc,distributes,VBZ
9,distributes,distribute,VBZ,conj,makes,VBZ


In [ ]:
def parse_compound_aspects(data):
    count = 0
    for i in range(len(data)):
        j = 0
        generated_aspect_words_to_be_deleted = set()
        while(j < len(data[i].generated_aspect_words)):
            generated_aspect_term = data[i].generated_aspect_words[j].aspect_term
            individual_aspect_terms = generated_aspect_term.split(" ")
            if(len(individual_aspect_terms) > 1):
                for individual_aspect_term in individual_aspect_terms:
                    # print(f"Generated: {data[i].generated_aspect_words}")
                    k = 0
                    while(k < len(data[i].generated_aspect_words)):
                        if k == j:
                            k += 1
                            continue
                        # print(f"i : {i} j : {j} k : {k} {individual_aspect_term}")
                        if individual_aspect_term == data[i].generated_aspect_words[k].aspect_term:
                            data[i].generated_aspect_words[j].sentiment_terms += (data[i].generated_aspect_words[k].sentiment_terms)
                            generated_aspect_words_to_be_deleted.add(data[i].generated_aspect_words[k])
                            # print(f"Compound: {generated_aspect_term}, Deleting {data[i].generated_aspect_words[k]}")
                            count += 1
                        k += 1
            j += 1
        for generated_aspect_word_to_be_deleted in generated_aspect_words_to_be_deleted:
            data[i].generated_aspect_words.remove(generated_aspect_word_to_be_deleted)
    return count

def add_or_update_aspect_word(sentence, aspect_term, sentiment_term):
    if sentiment_term == None:
        sentence.generated_aspect_words.append(AspectWord(aspect_term, []))
        return True
    for i in range(len(sentence.generated_aspect_words)):
        if sentence.generated_aspect_words[i].aspect_term == aspect_term:
            sentence.generated_aspect_words[i].sentiment_terms.append(sentiment_term)
            return True

    sentence.generated_aspect_words.append(AspectWord(aspect_term, [sentiment_term]))
    return True

def generate_aspect_words(data):
    dependencies= {"nsubj":[],"amod":[],"dobj":[],"nmod":[],"acl":[],"conj":[],"compound":[]}
        
    for i in range(len(data)):
        extracted_attributes_df = extract_attributes(data[i].sentence)
        data[i].generated_aspect_words.clear()

        # Rules:
        for j in extracted_attributes_df.index:
            
            # Bell, based in Los Angeles, makes and distributes electronic, computer and building products.
            if extracted_attributes_df['dep'][j] == "nsubj":
                # Example: (best, display)
                if extracted_attributes_df['gov_pos'][j] in ["JJ", "JJR", "JJS"] and extracted_attributes_df['pos'][j] in ["NN", "NNS", "NNP"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['word'][j], extracted_attributes_df['gov'][j])
                # Example: (distributes, XYZ Company)
                elif extracted_attributes_df['gov_pos'][j] in ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"] and extracted_attributes_df['pos'][j] in ["NN", "NNS", "NNP"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['word'][j], extracted_attributes_df['gov'][j])
                # dependencies["nsubj"].append(((extracted_attributes_df['word'][j],extracted_attributes_df['pos'][j]),(extracted_attributes_df['gov'][j],extracted_attributes_df['gov_pos'][j])))
            
            elif extracted_attributes_df['dep'][j] == "amod":
                # Example: (products, electronic)
                if extracted_attributes_df['gov_pos'][j] in ["NN", "NNS", "NNP"] and extracted_attributes_df['pos'][j] in ["JJ", "JJR", "JJS"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['gov'][j], extracted_attributes_df['word'][j])
                # Example: (products, building)
                elif extracted_attributes_df['gov_pos'][j] in ["NN", "NNS", "NNP"] and extracted_attributes_df['pos'][j] in ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['gov'][j], extracted_attributes_df['word'][j])
                # dependencies["amod"].append(((extracted_attributes_df['gov'][j],extracted_attributes_df['gov_pos'][j]),(extracted_attributes_df['word'][j],extracted_attributes_df['pos'][j])))
            
            elif extracted_attributes_df['dep'][j] == "obj":
                # Example: (makes, products)
                if extracted_attributes_df['gov_pos'][j] in ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"] and extracted_attributes_df['pos'][j] in ["NN", "NNS", "NNP"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['word'][j], extracted_attributes_df['gov'][j])
                    dependencies["dobj"].append(((extracted_attributes_df['word'][j],extracted_attributes_df['pos'][j]),(extracted_attributes_df['gov'][j],extracted_attributes_df['gov_pos'][j])))

            elif extracted_attributes_df['dep'][j] == "nmod":
                # Both are aspects
                # Example: (display, computer)
                if extracted_attributes_df['gov_pos'][j] in ["NN"] and extracted_attributes_df['pos'][j] in ["NN", "NS"]:
                    # Check which before
                    add_or_update_aspect_word(data[i], extracted_attributes_df['word'][j] + " " + extracted_attributes_df['gov'][j], None)
                    # dependencies["nmod"].append(((extracted_attributes_df['word'][j] + " " + extracted_attributes_df['gov'][j],extracted_attributes_df['gov_pos'][j],extracted_attributes_df['pos'][j])))

                elif extracted_attributes_df['gov_pos'][j] in ["JJ"] and extracted_attributes_df['pos'][j] in ["NN"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['word'][j], extracted_attributes_df['gov'][j])
                    # dependencies["nmod"].append(((extracted_attributes_df['word'][j],extracted_attributes_df['pos'][j]),(extracted_attributes_df['gov'][j],extracted_attributes_df['gov_pos'][j])))

            elif extracted_attributes_df['dep'][j] == "acl":
                if extracted_attributes_df['gov_pos'][j] in ["NN"] and extracted_attributes_df['pos'][j] in ["JJ"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['gov'][j], extracted_attributes_df['word'][j])
                
                elif extracted_attributes_df['gov_pos'][j] in ["NNS"] and extracted_attributes_df['pos'][j] in ["VBP"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['gov'][j], extracted_attributes_df['word'][j])
                # dependencies["acl"].append(((extracted_attributes_df['gov'][j],extracted_attributes_df['gov_pos'][j]),(extracted_attributes_df['word'][j],extracted_attributes_df['pos'][j])))

            elif extracted_attributes_df['dep'][j] == "conj":
                # Both are aspects
                if extracted_attributes_df['gov_pos'][j] in ["NN"] and extracted_attributes_df['pos'][j] in ["NN", "NNS", "NNP"]:
                    # Check which before
                    add_or_update_aspect_word(data[i], extracted_attributes_df['word'][j] + " " + extracted_attributes_df['gov'][j], None)
                    # dependencies["conj"].append(((extracted_attributes_df['word'][j] + " " + extracted_attributes_df['gov'][j],extracted_attributes_df['gov_pos'][j],extracted_attributes_df['pos'][j])))

                elif extracted_attributes_df['gov_pos'][j] in ["NN", "NNS", "NNP"] and extracted_attributes_df['pos'][j] in ["JJ"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['gov'][j], extracted_attributes_df['word'][j])
                    # dependencies["conj"].append(((extracted_attributes_df['gov'][j],extracted_attributes_df['gov_pos'][j]),(extracted_attributes_df['word'][j],extracted_attributes_df['pos'][j])))

                elif extracted_attributes_df['gov_pos'][j] in ["NN", "NNS", "NNP"] and extracted_attributes_df['pos'][j] in ["VBZ"]:
                    add_or_update_aspect_word(data[i], extracted_attributes_df['gov'][j], extracted_attributes_df['word'][j])
                    # dependencies["conj"].append(((extracted_attributes_df['gov'][j],extracted_attributes_df['gov_pos'][j]),(extracted_attributes_df['word'][j],extracted_attributes_df['pos'][j])))

            elif extracted_attributes_df['dep'][j] == "compound":
                # Both are aspects
                if extracted_attributes_df['gov_pos'][j] in ["NN"] and extracted_attributes_df['pos'][j] in ["NN"]:
                    # Check which before
                    add_or_update_aspect_word(data[i], extracted_attributes_df['word'][j] + " " + extracted_attributes_df['gov'][j], None)
                    # dependencies["compound"].append(((extracted_attributes_df['word'][j] + " " + extracted_attributes_df['gov'][j],extracted_attributes_df['gov_pos'][j],extracted_attributes_df['pos'][j])))
    parse_compound_aspects(data)
    return dependencies

In [ ]:
train_dependencies = generate_aspect_words(laptop_train_data)
laptop_train_data_df = data_list_to_df(laptop_train_data, aspect_terms = "generated", aspect_polarity = True)
laptop_train_data_df.head()

,ID,Sentence,Generated Aspect Terms,Generated Sentiment Terms,Generated Polarities
0,611,Not only are the versions of these programs ab...,[versions],"[[worked, able, superior]]",[None]
1,260,Sometimes you really have to tap the pad to ge...,[pad],[[tap]],[None]
2,799,"-When battery life went to 4 hours or less, to...",[battery life],[[went]],[None]
3,397,If you don't feel comfortable doing it yoursel...,"[case, one, Buy]","[[buy], [white, bought], [Best]]","[None, None, None]"
4,834,/ awesome cooling system/ much better grafics ...,"[cooling system, card, GB RAM, RAM screen, bac...","[[awesome], [better], [], [LED], [LED], [LED, ...","[None, None, None, None, None, None]"


In [ ]:
# import sys

# orig_stdout = sys.stdout
# f = open(root_dir + train_folder + "train_dependencies.txt", 'w')
# sys.stdout = f

# for k,v in train_dependencies.items():
#     print(f"{k}\n")
#     for x in v:
#         print(f"\t{x}")

# sys.stdout = orig_stdout
# f.close()

In [ ]:
def match_aspect_terms(actual_aspect_term, generated_aspect_term):
    n = len(generated_aspect_term.split(" "))
    for i in range(n):
        temp = generated_aspect_term.split(" ",maxsplit = i)
        if len(temp) > 1:
            rest = temp[1]
        else:
            rest = temp[0]
        if re.search(rest, actual_aspect_term):
            return len(rest)/len(actual_aspect_term)
        generated_aspect_term = rest
    return 0

In [ ]:
def aspect_term_performance(data):
    true_positives = 0
    false_positives = 0
    no_of_actual_aspect_terms = 0
    no_of_generated_aspect_terms = 0

    for i in range(len(data)):
        no_of_generated_aspect_terms += len(data[i].generated_aspect_words)
        no_of_actual_aspect_terms += len(data[i].actual_aspect_words)
        for generated_aspect_word in data[i].generated_aspect_words:
            found = False
            for actual_aspect_word in data[i].actual_aspect_words:
                matched_part = match_aspect_terms(actual_aspect_word.aspect_term, generated_aspect_word.aspect_term)
                if matched_part > 0:
                    true_positives += matched_part
                    found = True
                    break
            if not found:
                false_positives += 1

    ratios = {"Aspect Term Extraction Precision": true_positives / no_of_generated_aspect_terms, "Aspect Term Extraction Recall":true_positives/no_of_actual_aspect_terms, "Aspect Term Extraction True Positives":true_positives, "Actual Aspect Terms":no_of_actual_aspect_terms, "Generated Aspect Terms":no_of_generated_aspect_terms}
    return ratios

In [ ]:
laptop_train_data_ratios = aspect_term_performance(laptop_train_data)
pd.DataFrame.from_dict(laptop_train_data_ratios, orient = 'index',columns = ["Parameters"])

,Parameters
Aspect Term Extraction Precision,0.419667
Aspect Term Extraction Recall,0.566810
Aspect Term Extraction True Positives,1191.433998
Actual Aspect Terms,2102.000000
Generated Aspect Terms,2839.000000


# Train the Model
Make a frequency dictionary for all the Sentiment Words

In [ ]:
def train(data):
    # Do preprocessing if needed
    sentiment_terms_frequency = {}
    for sentence in data:
        for generated_aspect_word in sentence.generated_aspect_words:
            polarity = None
            for actual_aspect_word in sentence.actual_aspect_words:
                matched_part = match_aspect_terms(actual_aspect_word.aspect_term, generated_aspect_word.aspect_term)
                if matched_part > 0: 
                    if actual_aspect_word.polarity == 'positive':
                        polarity = 1
                    
                    elif actual_aspect_word.polarity == 'negative':
                        polarity = -1
                    
                    else:
                        polarity = 0
                    break    
            if polarity ==  None:
                continue
            for sentiment_term in generated_aspect_word.sentiment_terms:
                if sentiment_term in list(sentiment_terms_frequency.keys()): 
                    sentiment_terms_frequency[sentiment_term]['frequency'] += 1
                    sentiment_terms_frequency[sentiment_term]['polarity'] += (polarity)
                
                else:
                    sentiment_terms_frequency[sentiment_term] = {'frequency':1, 'polarity':polarity}

    for key in list(sentiment_terms_frequency.keys()):
        sentiment_terms_frequency[key] = (sentiment_terms_frequency[key]['polarity']/sentiment_terms_frequency[key]['frequency'])

    return sentiment_terms_frequency

In [ ]:
laptop_sentiment_terms_frequency_train = train(laptop_train_data)
print(f"Number of Sentiment Terms: {len(laptop_sentiment_terms_frequency_train)}")
df = pd.DataFrame.from_dict(laptop_sentiment_terms_frequency_train, orient = 'index', columns =["Polarity"])
df.head()

Number of Sentiment Terms: 615


,Polarity
tap,-1.000000
went,-0.833333
buy,-0.333333
awesome,1.000000
better,0.142857


# Test the Model

In [ ]:
def predict(data, sentiment_terms_frequency):
    generate_aspect_words(data)
    unk_words = set() 
    for i in range(len(data)):
        for j in range(len(data[i].generated_aspect_words)):
            sentiment_terms = data[i].generated_aspect_words[j].sentiment_terms
            unk = 0
            polarity = 0
            for term in sentiment_terms:
                if term not in sentiment_terms_frequency.keys():
                    unk += 1
                    unk_words.add(term)
                else:
                    polarity += sentiment_terms_frequency[term]
            if (len(sentiment_terms) - unk != 0):
                data[i].generated_aspect_words[j].polarity = polarity/(len(sentiment_terms)-unk)
            else:
                data[i].generated_aspect_words[j].polarity = 0
    return len(unk_words)

def find_polarity(value, threshold = 0.1):
    if (value < -1*threshold):
        return "negative"
    elif (value > threshold):
        return "positive"
    else:
        return "neutral"

def classify(data, sentiment_terms_frequency):
    no_of_unk_words = predict(data, sentiment_terms_frequency)
    for i in range(len(data)):
        for j in range(len(data[i].generated_aspect_words)):
            data[i].generated_aspect_words[j].polarity = find_polarity(data[i].generated_aspect_words[j].polarity)
    return no_of_unk_words

In [ ]:
laptop_test_1_data = xml_to_sentences(root_dir + test_1_folder + laptop_test_1_file, data_type = "Test")
laptop_test_1_data_df = data_list_to_df(laptop_test_1_data, aspect_terms = "none", aspect_polarity = False)
laptop_test_1_data_df.head()

,ID,Sentence
0,1,"Boot time is super fast, around anywhere from ..."
1,2,tech support would not fix the problem unless ...
2,3,but in resume this computer rocks!
3,4,Set up was easy.
4,5,Did not enjoy the new Windows 8 and touchscree...


In [ ]:
laptop_test_1_predictions_unk_count = classify(laptop_test_1_data, laptop_sentiment_terms_frequency_train)
print("Number of Unknown Sentiment Terms:", laptop_test_1_predictions_unk_count)
laptop_test_1_data_df = data_list_to_df(laptop_test_1_data, aspect_terms = "generated", aspect_polarity = True)
laptop_test_1_data_df.head()

Number of Unknown Sentiment Terms: 279


,ID,Sentence,Generated Aspect Terms,Generated Sentiment Terms,Generated Polarities
0,1,"Boot time is super fast, around anywhere from ...",[Boot time],[[fast]],[positive]
1,2,tech support would not fix the problem unless ...,"[tech support, problem, plan]","[[fix], [fix], [bought]]","[negative, negative, negative]"
2,3,but in resume this computer rocks!,[computer],[[rocks]],[neutral]
3,4,Set up was easy.,[],[],[]
4,5,Did not enjoy the new Windows 8 and touchscree...,"[functions, Windows]","[[new, enjoy], [enjoy]]","[neutral, neutral]"


# Performance of Sentiment Classification

In [ ]:
# Split data and do this
def classification_performance(data):
    matches = 0
    non_matches = 0
    for i in range(len(data)):
        for actual_aspect_word in data[i].actual_aspect_words:
            actual_aspect_term = actual_aspect_word.aspect_term
            actual_polarity = actual_aspect_word.polarity
            for generated_aspect_word in data[i].generated_aspect_words:
                if (match_aspect_terms(actual_aspect_term, generated_aspect_word.aspect_term) > 0):
                    if (actual_polarity == generated_aspect_word.polarity):
                        matches+=1
                    else:
                        non_matches+=1
                    break

    return {"Sentiment Matches":matches, "Sentiment Non-Matches":non_matches, "Sentiment Classification Precision":matches/(matches+non_matches)}

In [ ]:
laptop_valid_data_predictions_unk_count = classify(laptop_valid_data, laptop_sentiment_terms_frequency_train)
print("Unknown Sentiment Terms:", laptop_valid_data_predictions_unk_count)
laptop_valid_data_df = data_list_to_df(laptop_valid_data, aspect_terms = "none", aspect_polarity = False)
laptop_valid_data_df.head()

Unknown Sentiment Terms: 95


,ID,Sentence
0,433,"WHEN TYPING, LETTERS AND SPACES ARE FREQUENTLY..."
1,59,I love the glass touchpad.
2,1021,Later it held zero charge and its replacemen...
3,678,This computer is exceptionally thin for it's s...
4,1319,It has just enough RAM to run smoothly and eno...


In [ ]:
laptop_valid_data_ratios = aspect_term_performance(laptop_valid_data)
laptop_valid_data_ratios.update(classification_performance(laptop_valid_data))
pd.DataFrame.from_dict(laptop_valid_data_ratios, orient = 'index',columns = ["Parameters"])

,Parameters
Aspect Term Extraction Precision,0.467085
Aspect Term Extraction Recall,0.607576
Aspect Term Extraction True Positives,155.539348
Actual Aspect Terms,256.000000
Generated Aspect Terms,333.000000
Sentiment Matches,87.000000
Sentiment Non-Matches,82.000000
Sentiment Classification Precision,0.514793


In [ ]:
def write_to_csv(data, path):
    data_df = data_list_to_df(data, aspect_terms = "both", aspect_polarity = True)
    data_df.to_csv(path_or_buf = path)

In [ ]:
# write_to_csv(laptop_train_data, root_dir + train_folder + "generated_laptop_train_data.csv")
# write_to_csv(laptop_test_1_data, root_dir + train_folder + "generated_laptop_test_1_data.csv")
# write_to_csv(laptop_valid_data, root_dir + train_folder + "generated_laptop_valid_data.csv")

# Demo


In [ ]:
def test(string):
  test_data = []
  test_data.append(string)
  test_predictions_unk_count = predict(test_data, mixed_sentiment_terms_frequency_train)
  print(test_data.head())
  classify(test_predictions, mixed_sentiment_terms_frequency_train)
  print("Unknown Sentiment Words:", test_predictions_unk_count)
  print("")
  print("Analysis:")
  print(test_predictions)

In [ ]:
test("It is a good laptop but bad mouse")

In [ ]:
test("It is super fast and has outstanding graphics.")

In [ ]:
test("Laptop good, mouse bad")

## Shortcomings and improvements

* Compound aspect terms
* Compount sentiment words
* More rules, curent ones don't handle many cases 
* Improve accuracy
* Precision of genration of aspect terms is an approximation
* Can try Ngram model - for 3 cases with aspect-aspect

